In [1]:
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime

# 만화 얼굴 PNG 이미지 로드
face_img_paths = [
    '/home/jm/dev_ws/mldl/src/face_0_default.png',
    '/home/jm/dev_ws/mldl/src/face_1_happy.png',
    '/home/jm/dev_ws/mldl/src/face_2_surprise.png',
]
face_imgs = [cv2.imread(p, cv2.IMREAD_UNCHANGED) for p in face_img_paths]
# 시청자 기준 왼쪽 기울임용 happy 좌우반전 준비 (cv2.flip(img, 1))
happy_flipped = cv2.flip(face_imgs[1], 1) if face_imgs[1] is not None else None

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=6,  # 여러 얼굴 처리
    refine_landmarks=True,
    min_detection_confidence=0.3,
    min_tracking_confidence=0.3,
)

mp_face_det = mp.solutions.face_detection
face_det = mp_face_det.FaceDetection(model_selection=1, min_detection_confidence=0.3)

# 얼굴 각도 계산 (양눈 기준 기울기, 좌우 뒤집힘 보정)
def get_face_angle(landmarks, frame_shape):
    left_eye = landmarks[33]
    right_eye = landmarks[263]
    lx, ly = int(left_eye.x * frame_shape[1]), int(left_eye.y * frame_shape[0])
    rx, ry = int(right_eye.x * frame_shape[1]), int(right_eye.y * frame_shape[0])
    angle = -np.degrees(np.arctan2(ry - ly, rx - lx))  # flip된 화면 보정
    return angle

# 얼굴 bbox, 중심, 크기 계산
def get_face_bbox(landmarks, frame_shape):
    points = np.array([(int(l.x * frame_shape[1]), int(l.y * frame_shape[0])) for l in landmarks])
    x, y, w, h = cv2.boundingRect(points)
    cx, cy = x + w // 2, y + h // 2
    return x, y, w, h, cx, cy

# FaceDetection 결과에서 bbox/각도 추출 (마스크/안경 fallback)
def bbox_angle_from_detection(det, frame_shape):
    rel_box = det.location_data.relative_bounding_box
    x = int(rel_box.xmin * frame_shape[1])
    y = int(rel_box.ymin * frame_shape[0])
    w = int(rel_box.width * frame_shape[1])
    h = int(rel_box.height * frame_shape[0])
    cx, cy = x + w // 2, y + h // 2
    if det.location_data.relative_keypoints:
        kp = det.location_data.relative_keypoints
        lx = int(kp[0].x * frame_shape[1])
        ly = int(kp[0].y * frame_shape[0])
        rx = int(kp[1].x * frame_shape[1])
        ry = int(kp[1].y * frame_shape[0])
        angle = -np.degrees(np.arctan2(ry - ly, rx - lx))
    else:
        angle = 0.0
    return (x, y, w, h, cx, cy), angle

# 얼굴 마스크 생성 (컨벡스 헐 + 부드러운 feather)
def build_skin_mask(landmarks, frame_shape):
    pts = np.array([(int(l.x * frame_shape[1]), int(l.y * frame_shape[0])) for l in landmarks], dtype=np.int32)
    hull = cv2.convexHull(pts)
    mask = np.zeros(frame_shape[:2], dtype=np.uint8)
    cv2.fillConvexPoly(mask, hull, 255)
    mask = cv2.GaussianBlur(mask, (51, 51), 0)
    return mask.astype(np.float32) / 255.0

# detection 기반 마스크 (랜드마크 없을 때 타원)
def build_ellipse_mask(bbox, frame_shape):
    x, y, w, h, cx, cy = bbox
    mask = np.zeros(frame_shape[:2], dtype=np.uint8)
    axes = (max(1, w // 2), max(1, h // 2))
    cv2.ellipse(mask, (cx, cy), axes, 0, 0, 360, 255, -1)
    mask = cv2.GaussianBlur(mask, (51, 51), 0)
    return mask.astype(np.float32) / 255.0

# 살구색으로 얼굴 채우기
def blend_skin(frame, mask, skin_color=(190, 220, 255)):
    color_img = np.full_like(frame, skin_color)
    mask_3d = mask[..., None]
    blended = frame * (1 - mask_3d) + color_img * mask_3d
    return blended.astype(np.uint8)

# 조건별 얼굴 PNG 선택 (최종 분기)
# - 가까이 있을 때: surprise
# - angle > 18: happy_flipped
# - angle < -18: happy 원본
# - 기본: default
def select_character(bbox, angle, frame_shape):
    _, _, w, h, _, _ = bbox
    if w > frame_shape[1] * 0.42 or h > frame_shape[0] * 0.42:
        return face_imgs[2]
    if angle > 18 and happy_flipped is not None:
        return happy_flipped
    if angle < -18:
        return face_imgs[1]
    return face_imgs[0]

# 얼굴 오버레이 (회전 + 알파) 통합
def overlay_character(canvas, bbox, angle, char_img, alpha=0.95, scale=1.12):
    if char_img is None:
        return canvas

    x, y, w, h, cx, cy = bbox
    H, W = canvas.shape[:2]
    if w <= 0 or h <= 0:
        return canvas

    target_w = max(1, int(w * scale))
    target_h = max(1, int(h * scale))
    char_resized = cv2.resize(char_img, (target_w, target_h), interpolation=cv2.INTER_AREA)
    M = cv2.getRotationMatrix2D((target_w // 2, target_h // 2), angle, 1.0)
    char_rotated = cv2.warpAffine(
        char_resized,
        M,
        (target_w, target_h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(0, 0, 0, 0),
    )

    x1 = int(cx - target_w / 2)
    y1 = int(cy - target_h / 2)
    x2 = x1 + target_w
    y2 = y1 + target_h

    x1c, y1c = max(0, x1), max(0, y1)
    x2c, y2c = min(W, x2), min(H, y2)
    if x1c >= x2c or y1c >= y2c:
        return canvas

    icon_crop = char_rotated[y1c - y1 : y2c - y1, x1c - x1 : x2c - x1]
    region = canvas[y1c:y2c, x1c:x2c].astype(np.float32)

    if icon_crop.shape[2] == 4:
        icon_rgb = icon_crop[:, :, :3].astype(np.float32)
        icon_alpha = (icon_crop[:, :, 3] / 255.0 * alpha)[..., None]
        blended = region * (1 - icon_alpha) + icon_rgb * icon_alpha
        canvas[y1c:y2c, x1c:x2c] = blended.astype(np.uint8)
    else:
        canvas[y1c:y2c, x1c:x2c] = icon_crop[:, :, :3]
    return canvas

# 간단한 각도 스무딩 버퍼 (사람별 인덱스 관리)
class AngleSmoother:
    def __init__(self, momentum=0.75):
        self.momentum = momentum
        self.value = None

    def update(self, angle):
        if self.value is None:
            self.value = angle
        else:
            self.value = self.momentum * self.value + (1 - self.momentum) * angle
        return self.value

# 여러 얼굴에 대한 스무더 풀
smoothers = {}

def get_smoother(idx, momentum=0.8):
    if idx not in smoothers:
        smoothers[idx] = AngleSmoother(momentum=momentum)
    return smoothers[idx]

# 간단한 중복 체크용 헬퍼 (센터 거리)
def is_near_existing(center, centers, thr):
    for c in centers:
        if np.linalg.norm(np.array(center) - np.array(c)) < thr:
            return True
    return False

# 실시간 웹캠 처리 + 녹화
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError('웹캠을 열 수 없습니다.')

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
recording = False
writer = None
save_path = None

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results_mesh = face_mesh.process(rgb)
        results_det = face_det.process(rgb)

        output = frame.copy()
        used_centers = []
        idx_counter = 0

        if results_mesh.multi_face_landmarks:
            for face_landmarks in results_mesh.multi_face_landmarks:
                bbox = get_face_bbox(face_landmarks.landmark, frame.shape)
                angle_raw = get_face_angle(face_landmarks.landmark, frame.shape)
                angle = get_smoother(idx_counter).update(angle_raw)
                skin_mask = build_skin_mask(face_landmarks.landmark, frame.shape)
                skin_frame = blend_skin(output, skin_mask)
                char_img = select_character(bbox, angle, frame.shape)
                output = overlay_character(skin_frame, bbox, angle, char_img, alpha=0.95, scale=1.12)
                used_centers.append((bbox[4], bbox[5]))
                idx_counter += 1

        if results_det.detections:
            for det in results_det.detections:
                bbox, angle_raw = bbox_angle_from_detection(det, frame.shape)
                cx, cy = bbox[4], bbox[5]
                thr = max(bbox[2], bbox[3]) * 0.35
                if is_near_existing((cx, cy), used_centers, thr):
                    continue
                angle = get_smoother(idx_counter).update(angle_raw)
                skin_mask = build_ellipse_mask(bbox, frame.shape)
                skin_frame = blend_skin(output, skin_mask)
                char_img = select_character(bbox, angle, frame.shape)
                output = overlay_character(skin_frame, bbox, angle, char_img, alpha=0.95, scale=1.12)
                used_centers.append((cx, cy))
                idx_counter += 1

        if recording and writer is not None:
            writer.write(output)
            cv2.putText(output, 'REC', (12, 32), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

        cv2.imshow('cartoon face overlay', output)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        if key == ord('r'):
            if not recording:
                ts = datetime.now().strftime('%Y%m%d_%H%M%S')
                save_path = f'cartoon_capture_{ts}.mp4'
                fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
                writer = cv2.VideoWriter(save_path, fourcc, fps, (output.shape[1], output.shape[0]))
                recording = True
                print(f'녹화 시작: {save_path}')
            else:
                recording = False
                if writer is not None:
                    writer.release()
                    print(f'녹화 종료: {save_path}')
                writer = None

finally:
    if writer is not None:
        writer.release()
    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1765435501.462362   64186 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765435501.466149   64246 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1765435501.470810   64239 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1765435501.472159   64186 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765435501.475097   64256 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1765435501.494355   64249 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765435501.496191 

```mermaid
flowchart TD
  A[Start/Init] --> B[Capture frame]
  B --> C{FaceMesh ok?}
  C -- yes --> D[Landmarks -> bbox/angle]
  C -- no --> E[FaceDetection -> bbox/angle]
  D --> F[Skin mask]
  E --> F
  F --> G[Select PNG by angle/size]
  G --> H[Rotate/Scale/Blend]
  H --> I[Display/Record]
  I --> B
```
